# Importing Libraries

In [ ]:
import gym # For Game Environment
import numpy as np # Numpy
from itertools import count # For Running the Counter
from collections import namedtuple

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

# Making Gym Environment

In [11]:
SavedAction= namedtuple('SavedAction', ['log_prob', 'value'])
env = gym.make('CartPole-v0')

In [12]:
# Defining Action and Critic 
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(4, 128) 
        self.actor = nn.Linear(128, 2)
        self.critic = nn.Linear(128, 1)
        self.saved_actions = []
        self.rewards = []
    def forward(self, x):
        x = F.relu(self.fc1(x))
        action_prob = F.softmax(self.actor(x), dim=-1)
        state_values = self.critic(x)
        return action_prob, state_values

In [13]:
# Creating Data how model Performed
def select_action(state):
    state = torch.from_numpy(state).float()
    probs, state_value = model(state)
    m = Categorical(probs)
    action = m.sample()
    model.saved_actions.append(SavedAction(m.log_prob(action), state_value))
    return action.item()

In [14]:
# Episode Training 
def finish_episode():
    R = 0
    saved_actions = model.saved_actions
    policy_losses = []
    value_losses =[]
    returns = []
    
    for r in model.rewards[::-1]:
        R = r + 0.99 * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    
    for (log_prob, value), R in zip(saved_actions, returns):
        advantage = R - value.item()
        
        policy_losses.append(-log_prob * advantage)
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))
    
    optimizer.zero_grad()
    loss = torch.stack(policy_losses).sum() + torch.stack(value_losses).sum()
    
    loss.backward()
    optimizer.step()
    
    del model.rewards[:]
    del model.saved_actions[:]

In [15]:
# Defining The Model
model = ActorCritic()
optimizer = optim.Adam(model.parameters(), lr=3e-2)
eps = np.finfo(np.float32).eps.item()

## Training The Model

In [16]:
def train():
    running_reward = 10
    for i_episode in count(): # We need around this much episodes
        state = env.reset()
        ep_reward = 0
        for t in range(1, 10000):
            action = select_action(state)
            state, reward, done, _ = env.step(action)
            model.rewards.append(reward)
            ep_reward += reward
            if done:
                break
        running_reward = 0.05 * ep_reward + (1-0.05) * running_reward
        finish_episode()
        if i_episode % 10 == 0: # We will print some things out
            print("Episode {}\tLast Reward: {:.2f}\tAverage reward: {:.2f}".format(
                i_episode, ep_reward, running_reward
            ))
        if running_reward > env.spec.reward_threshold:
            print("Solved, running reward is now {} and the last episode runs to {} time steps".format(
                    running_reward, t
            ))
            break

In [17]:
train()

Episode 0	Last Reward: 24.00	Average reward: 10.70
Episode 10	Last Reward: 18.00	Average reward: 11.60
Episode 20	Last Reward: 20.00	Average reward: 12.42
Episode 30	Last Reward: 18.00	Average reward: 25.20
Episode 40	Last Reward: 10.00	Average reward: 20.49
Episode 50	Last Reward: 32.00	Average reward: 21.82
Episode 60	Last Reward: 88.00	Average reward: 47.88
Episode 70	Last Reward: 67.00	Average reward: 63.06
Episode 80	Last Reward: 60.00	Average reward: 69.62
Episode 90	Last Reward: 97.00	Average reward: 76.06
Episode 100	Last Reward: 103.00	Average reward: 82.13
Episode 110	Last Reward: 81.00	Average reward: 100.90
Episode 120	Last Reward: 169.00	Average reward: 114.11
Episode 130	Last Reward: 141.00	Average reward: 118.45
Episode 140	Last Reward: 111.00	Average reward: 124.46
Episode 150	Last Reward: 117.00	Average reward: 128.53
Episode 160	Last Reward: 200.00	Average reward: 143.09
Episode 170	Last Reward: 200.00	Average reward: 165.60
Episode 180	Last Reward: 200.00	Average rew

# Visualizing How Well our model Performed

In [20]:
done = False
cnt = 0
observation = env.reset()
while True:
    cnt += 1
    env.render()
    action = select_action(observation)
    observation, reward, done, _ = env.step(action)
    if done:
        break
    # Lets see how long it lasts until failing
print(f"Game lasted {cnt} moves")



Game lasted 200 moves
